# Lab Three: Multi-modal and Multi-task

#### CS8321: Neural Networks and Machine Learning
    
Johnathan Barr - 39854013
<br>
Will Lacey - 45906124

## Lab Description
<em>In this lab, you will implement a multi-task network (possibly multi-modal) that classifies interactions between compounds/ ligands with proteins in the ChEMBL database (https://www.ebi.ac.uk/chembl/). The objective is to classify which ligands bind to which targets. Each target will be a separate task working from a  shared ligand representation. </em>

## Import Modules and Initialization

Before we begin, let's import essential packages for data analysis.

In [129]:
import numpy as np
import pandas as pd

## Downloading the ChEMBL Dataset

<em>[<strong>10 points</strong>] Download the ChEMBL database or setup queries to download a subset of the database. An already processed version of the dataset is available here (thanks to Niraj Verma): https://smu.box.com/s/smqmwlef0yehpieicwxqdr99k7f9ru04</em>

In [ ]:
# export data as .npy
f = open('./data.csv', 'r', encoding="ISO-8859-1")
lines = f.readlines()
f.close()


flis = [] # will have [id, ic50_val, unit, target, smiles] 

for line in lines[1:]: # first line contains the headers (therefore skiped)
    lis = line.strip().split(',')

    if len(lis) < 36:
        continue
    
    
    #print (lis[0], lis[3], lis[8], lis[36])
    # [id, ic50_val, unit, target, smiles]

    flis.append([lis[0], lis[3], lis[4], lis[8], lis[36]])
    #break

# I have no idea why python adds \x00 to each string
# Therefoe I removed it

for i in range (len(flis)):
    flis[i] = [j.replace('\x00','') for j in flis[i]]
    flis[i] = [j.replace('"','') for j in flis[i]]

# Some of the data have smiles or IC50 missing
# So I removed them as well
list = []
for i in flis:
    if len(i[1]) != 0 and len(i[-1]) != 0:
        #print (i)
        list.append(i)

X, t, y = [], [], []
for i in list:
    if i[2] == 'nM':
        if len(i[-1]) == 0 or len(i[-2]) ==0 or len(i[1]) ==0:
            print ('Thers a problem !!')
        X.append(i[-1])
        t.append(i[-2])
        y.append(i[1])

# smiles is the string representation of each ligand
# target is the protein where ligand binds (treat it as different schools)
# ic50 is the score
np.save('data/data.npy', {'smiles':X, 'target':t, 'ic50':y})

In [130]:
data = np.load('./data.npy', allow_pickle=True)

## Filtering the Top 100 Targets

<em>[<strong>10 points</strong>] Filter the database to the top 100 targets in the database. You will need a definition of "top" such as the targets with the most assays. From these top 100 targets, save the ligands that have an assay result for each of the targets. </em>

In [131]:
df = pd.DataFrame(data.tolist())

In [132]:
counts_series = df['target'].value_counts()

In [133]:
top_targets = counts_series.to_frame()[0:100].index.values.tolist()

In [134]:
top_data = df[df['target'].isin(top_targets)]

In [135]:
top_data

,smiles,target,ic50
422,COc1ccccc1CNC(=O)CCN2C(=O)Nc3ccccc3C2=O,CHEMBL3436040,12000
423,Cl.COc1ccc2nccc(C(O)C3CC4(CCN3CC4)C=C)c2c1,CHEMBL3436040,12000
424,Cn1c(nc2ccccc12)C(=O)c3cccc4ccccc34,CHEMBL3436040,12000
425,Cl.CN1C(=O)N(C)c2cc(CNCCNc3ccnc4cc(Cl)ccc34)ccc12,CHEMBL3436040,12000
426,COc1ccc(CNC(=O)C2=CC3=C(N=C4C=CC=CN4C3=O)N(CCc...,CHEMBL3436040,12000
...,...,...,...
747440,1.76,CHEMBL3887886,66
747446,9606,CHEMBL3887679,9.9
747457,S=P(N1CC1)(N2CC2)N3CC3,CHEMBL4017550,1000000
747458,CN(C)S(=O)(=O)c1ccc2Sc3ccccc3\C(=C\CCN4CCN(C)C...,CHEMBL4017550,30400


## Binarize the Binding Affinity

<em>[<strong>5 points</strong>] Binarize the binding affinity for each ligand in the assay. That is, convert the continuous measure of binding to binary. You should use the column 'IC50' for this calculation. Anything below 300 nM should be considered as an active binding. Anything above 10 uM should be considered non-binding (inactive). </em>

In [136]:
def binarize_affinity(value):
    value = float(value)
    if value < 300.0:
        return 1
    elif value > 10000.0:
        return 0
    else:
        return None

In [137]:
top_data['ic50'] = top_data['ic50'].apply(binarize_affinity)
top_data = top_data[top_data['ic50'].notna()]

/Users/Johnathan/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
top_data

,smiles,target,ic50
422,COc1ccccc1CNC(=O)CCN2C(=O)Nc3ccccc3C2=O,CHEMBL3436040,0.0
423,Cl.COc1ccc2nccc(C(O)C3CC4(CCN3CC4)C=C)c2c1,CHEMBL3436040,0.0
424,Cn1c(nc2ccccc12)C(=O)c3cccc4ccccc34,CHEMBL3436040,0.0
425,Cl.CN1C(=O)N(C)c2cc(CNCCNc3ccnc4cc(Cl)ccc34)ccc12,CHEMBL3436040,0.0
426,COc1ccc(CNC(=O)C2=CC3=C(N=C4C=CC=CN4C3=O)N(CCc...,CHEMBL3436040,0.0
...,...,...,...
747438,2.11,CHEMBL3887886,1.0
747440,1.76,CHEMBL3887886,1.0
747446,9606,CHEMBL3887679,1.0
747457,S=P(N1CC1)(N2CC2)N3CC3,CHEMBL4017550,0.0


## Featurize each Ligand
<em>[<strong>10 points</strong>] Featurize each ligand using RDKit (https://www.rdkit.org). This will convert the ligand representation into a binary vector of features. Mention any hyper parameters you use. </em>

In [29]:
# Import RDKit
from rdkit import Chem
import binascii

smile = 'COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC'
m = Chem.MolFromSmiles(smile)
smile_bytes = m.ToBinary()
smile_hex = binascii.b2a_hex(smile_bytes)
smile_hex_string = smile_hex.hex()
int(smile_hex_string, 16)

6422389453263301842435119605114597561215381584639000317383066857215430954801815638423704159150204986214843892165201567896652559075617608773145046279837853452245105309822039487459287682853439148491437694985912943477040180995333476906988784468658522230645329940683625055809862685508537186984049519458642377785886135296256988801306222335084054480461327101965335691042599001200551991479821973787410890783745482140827419270695519264044015282228097144606434465852974325686820928128972001406223820632319315370228196683546664017688686371240175875865099077076442821451352920232081363539912816235088564079278847011779731980798318186866671425908219408320656167823388022089612072555290144451025135952957560487741180537828613563313654133964504230272657113942878259625101113341994696900095757285900509047169197101771885798315825390828969349371961124508464564167633155641305042715411606667065669330254074610131915645319966361323800787300232913092014738175598126681613839015756229393328950584747982341873221888799721

In [14]:
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os

fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

In [15]:
feats = factory.GetFeaturesForMol(m)
len(feats)

13

In [16]:
feats

(<rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature at 0x12edeb570>,
 <rdkit.Chem.rdMolChemicalFeatures.MolChemicalFeature at 0x12edee2d0>)

## Train a multi-task model

<em>[<strong>20 points</strong>] Train a multi-task model (one model for each target). Use an 80/20 split for each target.</em>

In [139]:
tasks = dict()

for target in top_targets:
    target_rows = top_data.loc[top_data['target'] == target]
    tasks[target] = Bunch(data   = target_rows.smiles,
                          labels = target_rows.ic50)

In [140]:
tasks['CHEMBL2114881']

{'data': 26995              COc1ccc(cc1OC)c2ncc3ccccn23.OC(=O)C(=O)O
 26997        COc1cccc(c1)c2nnc(SCC(=O)c3ccc(O)c(O)c3)n2CC=C
 27001                  COc1ccc(cc1OC)c2c[nH]nc2c3ccc(O)cc3O
 27003                                 COc1cc(N)c2ncccc2c1SC
 27672     COc1cc(\C=C\C(=O)OCC(=O)N2CC3(C)CC2CC(C)(C)C3)...
                                 ...                        
 710330    CC1=CN([C@H]2C[C@H](O)[C@@H](CNCC3=Cc4cc(O)ccc...
 710383    COc1cc(\C=C\C(=O)OCC(=O)NC2=C(C)N(C)N(C2=O)c3c...
 710385    COc1cc(\C=C\C(=O)OCC(=O)N2CCC(Cc3ccccc3)CC2)ccc1O
 710395                           COc1cccc(OC)c1OCCCCN2CCCC2
 710396                   COc1ccc2NC(C)(C)C3=C(C(=S)SS3)c2c1
 Name: smiles, Length: 2026, dtype: object, 'labels': 26995     0.0
 26997     0.0
 27001     0.0
 27003     0.0
 27672     0.0
          ... 
 710330    0.0
 710383    0.0
 710385    0.0
 710395    0.0
 710396    0.0
 Name: ic50, Length: 2026, dtype: float64}

## Report the results

<em>[<strong>20 points</strong>] Report the results using AUC, BEDROC, and Enrichment factor. These metrics are easily calculated using the RDKit scoring library. Discuss the results (you will need to look up each evaluation metric to interpret the result). You may be interested in page 6 of the following document: https://www.dropbox.com/s/6je37ml475vg3ep/Srinivas2018ImplictDescriptorUnderReview.pdf?dl=0 </em>


In [25]:
# look at 4.2 in the document

## Additional Analysis

<em>[<strong>10 points</strong>] Finally, you have free reign to perform any other analysis. A suggested analysis is to add an additional mode of input data such as another fingerprint for each ligand.</em>